In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**get data**

In [2]:
data_link = "/content/drive/MyDrive/largeProject/code/NER/id_ner.csv"

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv(data_link)

In [5]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,tokens,ner_tags
0,"['Bạn', 'có_thể', 'tường_tận', 'thêm', 'về', '...","[8, 8, 8, 8, 8, 8, 8, 8, 1, 10, 10, 10, 10, 10..."
1,"['T', 'có', 'một', 'câu', 'hỏi', 'là', 'tháng'...","[8, 8, 8, 8, 8, 8, 8, 5, 11, 1, 10, 10, 10, 10..."
2,"['Cho', 'tui', 'biết', 'tình_hình', 'thực_hiện...","[8, 8, 8, 8, 8, 8, 1, 10, 9, 5, 11]"
3,"['Bạn', 'hãy', 'giúp', 'tui', 'giải_đáp', 'thắ...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 10, 7,..."
4,"['T', 'có', 'một', 'câu', 'hỏi', 'là', 'tháng'...","[8, 8, 8, 8, 8, 8, 8, 5, 11, 1, 10, 10, 10, 10..."


In [6]:
df['tokens'] = df['tokens'].map(lambda x : eval(x))
df['ner_tags'] = df['ner_tags'].map(lambda x : eval(x))

In [7]:
len(df['tokens'][0]) == len(df['ner_tags'][0])

True

In [8]:
import pickle

with open('/content/drive/MyDrive/largeProject/code/NER/l2i.pkl','rb') as f:
  l2i = pickle.load(f)

with open('/content/drive/MyDrive/largeProject/code/NER/i2l.pkl','rb') as f:
  i2l = pickle.load(f)

In [9]:
l2i

{'B_<năm-m>': 0,
 'B_<tên chỉ tiêu>': 1,
 'B_<đơn vị>': 2,
 'I_<tên cụm chỉ tiêu>': 3,
 'B_<tên cụm chỉ tiêu>': 4,
 'B_<tháng>': 5,
 'I_<đơn vị>': 6,
 'B_<quý>': 7,
 'O': 8,
 'B_<tên tổng công ty>': 9,
 'I_<tên chỉ tiêu>': 10,
 'B_<năm>': 11,
 'B_<tháng+1>': 12}

In [10]:
i2l

{0: 'B_<năm-m>',
 1: 'B_<tên chỉ tiêu>',
 2: 'B_<đơn vị>',
 3: 'I_<tên cụm chỉ tiêu>',
 4: 'B_<tên cụm chỉ tiêu>',
 5: 'B_<tháng>',
 6: 'I_<đơn vị>',
 7: 'B_<quý>',
 8: 'O',
 9: 'B_<tên tổng công ty>',
 10: 'I_<tên chỉ tiêu>',
 11: 'B_<năm>',
 12: 'B_<tháng+1>'}

**Dataset**

In [11]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.4 MB/s eta 0:00:00


In [12]:
import transformers

In [13]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 16
BASE_MODEL_PATH = "vinai/phobert-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(BASE_MODEL_PATH, do_lower_case=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
EPOCHS = 10

**Dataset**

In [15]:
import torch

class EntityDataset:
    def __init__(self, texts, tags):
        self.texts = texts
        self.tags = tags

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        tags = self.tags[item]

        ids = []
        target_tag = []

        for i, s in enumerate(text):
            inputs = tokenizer.encode(s, add_special_tokens=False)
            input_len = len(inputs)
            ids.extend(inputs)
            target_tag.extend([tags[i]] * input_len)

        ids = ids[:MAX_LEN - 2]
        target_tag = target_tag[:MAX_LEN - 2]

        ids = [0] + ids + [2] # : 0, : 2, : 1
        target_tag = [12] + target_tag + [12] # O: 12

        mask = [1] * len(ids) # 1: not masked, 0: masked
        token_type_ids = [0] * len(ids)

        padding_len = MAX_LEN - len(ids)

        ids = ids + ([1] * padding_len) # padding
        mask = mask + ([0] * padding_len) # 1: not masked, 0: masked
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_tag = target_tag + ([12] * padding_len) # O: 12

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
        }

**Model**

In [16]:
def loss_fn(output, target, mask, num_labels):
    lfn = torch.nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    loss = lfn(active_logits, active_labels)
    return loss


class EntityModel(torch.nn.Module):
    def __init__(self, num_tag):
        super(EntityModel, self).__init__()
        self.num_tag = num_tag
        self.bert = transformers.AutoModel.from_pretrained(BASE_MODEL_PATH,return_dict=False)
        # unfreeze bert
        for param in self.bert.parameters():
            param.requires_grad = False
        self.bert_drop = torch.nn.Dropout(0.3)
        self.out_tag = torch.nn.Linear(768, self.num_tag)

    def forward(self, ids, mask, token_type_ids, target_tag):
        o, _ = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        bo_tag = self.bert_drop(o)
        tag = self.out_tag(bo_tag)
        loss = loss_fn(tag, target_tag, mask, self.num_tag)

        return tag, loss

**Function training**

In [21]:
from tqdm import tqdm

def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        optimizer.zero_grad()
        _, loss = model(**data)
        # print(loss)
        loss.backward()
        optimizer.step()
        scheduler.step()
        final_loss += loss.item()
    return final_loss / len(data_loader)


def eval_fn(data_loader, model, device):
    model.eval()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        _, loss = model(**data)
        final_loss += loss.item()
    return final_loss / len(data_loader)

**Train,val,test**

In [17]:
import pandas as pd
import numpy as np

import joblib
import torch

from sklearn import preprocessing
from sklearn import model_selection

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import KFold

In [18]:
sentences = df["tokens"].to_list()
tag = df["ner_tags"].to_list()

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sentences, tag, test_size=0.05, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=1)

In [20]:
num_tag = len(i2l)

**Not k-fold**

In [37]:
train_sentences = X_train
train_tag = y_train
test_sentences = X_val
test_tag = y_val

train_dataset = EntityDataset(texts=train_sentences, tags=train_tag)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)

test_dataset = EntityDataset(texts=test_sentences, tags=test_tag)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = EntityModel(num_tag=num_tag)
model.load_state_dict(torch.load("/content/drive/MyDrive/largeProject/code/NER/phobert_fold1.pth"))
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
          {
              "params": [
                  p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
              ],
              "weight_decay": 0.001,
          },
          {
              "params": [
                  p for n, p in param_optimizer if any(nd in n for nd in no_decay)
              ],
              "weight_decay": 0.0,
          },
      ]

num_train_steps = int(len(train_sentences) / TRAIN_BATCH_SIZE * EPOCHS)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

epoch_id = 1
best_loss = np.inf

MODEL_PATH = '/content/phobert_fold1.pth'

for epoch in range(EPOCHS):
    train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
    test_loss = eval_fn(test_data_loader, model, device)
    print(f"Epoch {epoch_id}: Train Loss = {train_loss} Valid Loss = {test_loss}")
    if test_loss < best_loss:
      torch.save(model.state_dict(), MODEL_PATH)
      best_loss = test_loss
    epoch_id += 1

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 357/357 [00:38<00:00,  9.16it/s]


Epoch 1: Train Loss = 0.4295406137178864 Valid Loss = 0.35136516797108475


100%|██████████| 357/357 [00:39<00:00,  9.11it/s]


Epoch 2: Train Loss = 0.41756994973887934 Valid Loss = 0.3405294389951797


100%|██████████| 357/357 [00:38<00:00,  9.17it/s]


...

Epoch 5: Train Loss = 0.3989910310320226 Valid Loss = 0.3229471545259492


100%|██████████| 357/357 [00:39<00:00,  9.05it/s]


Epoch 6: Train Loss = 0.3957697498819494 Valid Loss = 0.3197976354540897


  1%|          | 17/1693 [00:08<14:11,  1.97it/s]


KeyboardInterrupt: ignored

**Kfold**

In [ ]:
num_tag = len(i2l)
kf = KFold(n_splits=5)

for train, test in kf.split(sentences):
  train_sentences = []
  test_sentences = []
  train_tag = []
  test_tag = []

  for id in train:
    train_sentences.append(sentences[id])
    train_tag.append(tag[id])

  for id in test:
    test_sentences.append(sentences[id])
    test_tag.append(tag[id])

  train_dataset = EntityDataset(texts=train_sentences, tags=train_tag)
  train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)

  test_dataset = EntityDataset(texts=test_sentences, tags=test_tag)
  test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1)

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model = EntityModel(num_tag=num_tag)

  model.to(device)

  param_optimizer = list(model.named_parameters())
  no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
  optimizer_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.001,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]

  num_train_steps = int(len(train_sentences) / TRAIN_BATCH_SIZE * EPOCHS)
  optimizer = AdamW(optimizer_parameters, lr=3e-5)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

  epoch_id = 1
  best_loss = np.inf

  MODEL_PATH = '/content/phobert_fold1.pth'

  for epoch in range(32):
      train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
      test_loss = eval_fn(test_data_loader, model, device)
      print(f"Epoch {epoch_id}: Train Loss = {train_loss} Valid Loss = {test_loss}")
      if test_loss < best_loss:
        torch.save(model.state_dict(), MODEL_PATH)
        best_loss = test_loss
      epoch_id += 1

**Inference**

In [52]:
def infer(sentence,num_tag=13):
  segmented_sentence = sentence.split(' ')
  test_dataset = EntityDataset(texts=[segmented_sentence], tags=[[0] * len(segmented_sentence)])
  # model = EntityModel(num_tag=num_tag)
  # model.load_state_dict(torch.load(MODEL_PATH))
  # model.to(device)

  tags = []
  with torch.no_grad():
    data = test_dataset[0]
    for k, v in data.items():
      data[k] = v.to(device).unsqueeze(0)
    tag, _ = model(**data)
    tags = []
    for ele in tag.argmax(2).cpu().numpy().reshape(-1):
      tags.append(i2l[ele])
    # predict_tag.append(tags[:len(sentence) + 2])

  decoded_sentence = []
  for tokenized_word in test_dataset[0]["ids"]:
    decoded_word = tokenizer.decode([tokenized_word])
    decoded_sentence.append(decoded_word)
  # print(decoded_sentence)
  # print(tags)
  print("{:15} {:5}".format("Word", "Tag"))
  print("="*30)
  for w, t in zip(decoded_sentence, tags):
    if w == "": break
    if w == "</s>": break
    print("{:15}:{:5}".format(w, t))

In [23]:
test_dataset = EntityDataset(texts=X_test, tags=y_test)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [39]:
model = EntityModel(num_tag=13)
model.load_state_dict(torch.load("/content/drive/MyDrive/largeProject/code/NER/phobert_fold2.pth"))

<All keys matched successfully>

In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

EntityModel(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(

In [41]:
pred_tag = []
with torch.no_grad():
  for data in tqdm(test_dataset):
    for k, v in data.items():
      data[k] = v.to(device).unsqueeze(0)
    tag, _ = model(**data)
    pred_tag.append(tag)

100%|██████████| 5700/5700 [01:21<00:00, 69.96it/s]


In [42]:
predict_tag = []
for tag, sentence in zip(pred_tag, X_test):
  tags = []
  for ele in tag.argmax(2).cpu().numpy().reshape(-1):
    tags.append(i2l[ele])
  predict_tag.append(tags[:len(sentence) + 2])

In [43]:
target_tag = []

for dataset, sentence in zip(test_dataset, X_test):
  tag = dataset['target_tag'][:len(sentence) + 2]
  tags = []
  for ele in tag:
    tags.append(i2l[int(ele)])
  target_tag.append(tags)

In [44]:
print(len(target_tag))
print(len(predict_tag))

5700
5700


In [31]:
!pip install sklearn_crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 11.7 MB/s eta 0:00:00


In [32]:
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder

In [45]:
new_target = []
for tag_lst in target_tag:
    for tag in tag_lst:
        new_target.append(tag)

new_predict = []
for tag_lst in predict_tag:
    for tag in tag_lst:
        new_predict.append(tag)

In [46]:
new_predict[:5]

['B_<tháng+1>', 'O', 'O', 'O', 'O']

In [47]:
new_target[:5]

['B_<tháng+1>', 'O', 'O', 'O', 'O']

In [48]:
print(classification_report(new_target, new_predict, labels=list(l2i.keys())))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

           B_<năm-m>       0.71      0.64      0.67       264
    B_<tên chỉ tiêu>       0.81      0.85      0.83      4374
          B_<đơn vị>       0.91      0.51      0.66       150
I_<tên cụm chỉ tiêu>       0.81      0.32      0.46      8253
B_<tên cụm chỉ tiêu>       0.73      0.58      0.65      1466
           B_<tháng>       0.78      0.92      0.84      4970
          I_<đơn vị>       0.00      0.00      0.00        22
             B_<quý>       0.93      0.98      0.96       784
                   O       0.92      0.99      0.95     48580
B_<tên tổng công ty>       0.90      0.97      0.94      3712
    I_<tên chỉ tiêu>       0.79      0.92      0.85     23092
             B_<năm>       0.93      0.97      0.95      4954
         B_<tháng+1>       1.00      0.82      0.90     20967

            accuracy                           0.88    121588
           macro avg       0.79      0.73      0.74    121588
      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Save model**

In [38]:
import shutil
shutil.copy('/content/phobert_fold1.pth', '/content/drive/MyDrive/largeProject/code/NER/phobert_fold2.pth')

'/content/drive/MyDrive/largeProject/code/NER/phobert_fold2.pth'

In [ ]:
model.load_state_dict(torch.load('/content/phobert_fold1.pth'))

<All keys matched successfully>

In [55]:
infer("tỉ lệ phản ánh khách hàng của tổng công ty VTS trong năm 2022 đạt bao nhiêu?")

Word            Tag  
<s>            :B_<tháng+1>
tỉ             :B_<tên chỉ tiêu>
lệ             :I_<tên cụm chỉ tiêu>
phản           :I_<tên chỉ tiêu>
ánh            :I_<tên cụm chỉ tiêu>
khách          :I_<tên chỉ tiêu>
hàng           :I_<tên chỉ tiêu>
của            :I_<tên cụm chỉ tiêu>
tổng           :O    
công           :I_<tên chỉ tiêu>
ty             :I_<tên chỉ tiêu>
V@@            :B_<tên tổng công ty>
TS             :B_<tên tổng công ty>
trong          :O    
năm            :O    
2022           :B_<năm>
đạt            :O    
bao            :O    
nhiê@@         :O    
u@@            :O    
?              :O    
